In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import Model
import keras_tuner

data_path = "./dataset"

physical_devices = tf.config.list_physical_devices("GPU")
print(f"GPUs: {len(physical_devices)}")

In [ ]:
img_width = 200
img_height = 100
batch_size = 32

train_data = keras.preprocessing.image_dataset_from_directory(
    data_path + "/training_set",
    label_mode="binary",  # one-hot encoding with two columns
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.25,
    subset="training",
    seed=0,
)
val_data = keras.preprocessing.image_dataset_from_directory(
    data_path + "/training_set",
    label_mode="binary",
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.25,
    subset="validation",
    seed=0,  # same seed as for training
)

test_data = keras.preprocessing.image_dataset_from_directory(
    data_path + "/test_set",
    label_mode="categorical",
    image_size=(img_width, img_height),
)

In [ ]:
def build_model(hp):
    model = keras.models.Sequential()
    model.add(keras.Input(shape=(img_width, img_height, 3)))

    model.add(keras.layers.Rescaling(1/255))
    model.add(keras.layers.Conv2D(16, 3, activation='relu', name='conv1'))
    model.add(keras.layers.Conv2D(16, 3, activation='relu', name='conv2'))
    model.add(keras.layers.MaxPooling2D(name='pool1'))
    model.add(keras.layers.Conv2D(64, 3, activation='relu', name='conv3'))
    model.add(keras.layers.Conv2D(64, 3, activation='relu', name='conv4'))
    model.add(keras.layers.MaxPooling2D(name='pool2'))

    model.add(keras.layers.Flatten())

    model.add(keras.layers.Dense(20, activation="relu"))
    model.add(keras.layers.Dense(20, activation="relu"))
    model.add(keras.layers.Dense(20, activation="relu"))

    model.add(keras.layers.Dense(1, activation="sigmoid", name="out"))


    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
tuner = keras_tuner.RandomSearch(build_model, objective="val_accuracy", max_trials=10,)
tuner.search(train_data, validation_data=val_data, epochs=20)

### Best Model

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
best_model = build_model(best_hp)
best_model.summary()

loss = []
val_loss = []
acc = []
val_acc = []
history = best_model.fit(train_data, epochs=15, validation_data=val_data)
loss.extend(history.history["loss"])
val_loss.extend(history.history["val_loss"])
acc.extend(history.history["val_accuracy"])
val_acc.extend(history.history["val_accuracy"])

In [ ]:
fig, ax = plt.subplots()
ax.plot(loss, "-b", label="training loss")
ax.plot(val_loss, "-r", label="validation loss")
ax.legend()
plt.show()

fig, ax = plt.subplots()
ax.plot(acc, "-b", label="training accuracy")
ax.plot(val_acc, "-r", label="validation accuracy")
ax.legend()
plt.show()

### Save Model

In [ ]:
best_model.save("fish_model")